### Setup

In [8]:
import numpy as np
import keras
import tensorflow as tf
from datetime import datetime
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split

keras.utils.set_random_seed(67)
np.random.seed(67)

### Dataset

In [2]:
# extra code – load and split the California housing dataset, like earlier
housing = fetch_california_housing()
xtrain, xtemp, ytrain, ytemp = train_test_split(
    housing.data, housing.target, random_state=67
)

xdev, xtest, ydev, ytest = train_test_split(xtemp, ytemp, random_state=67)

### Network

In [5]:
NORM_LAYER = keras.layers.Normalization()

NET = keras.Sequential(
    [
        keras.layers.Input(shape=(xtrain.shape[1:])),
        NORM_LAYER,
        keras.layers.Dense(30, activation="relu"),
        keras.layers.Dense(30, activation="relu"),
        keras.layers.Dense(1),
    ]
)

optimizer = keras.optimizers.SGD(learning_rate=1e-3)

NET.compile(loss="mse", optimizer=optimizer, metrics=["RootMeanSquaredError"])

In [6]:
NORM_LAYER.adapt(xtrain)

log_dir = "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")

tensorboard_callback = keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

H = NET.fit(
    xtrain,
    ytrain,
    epochs=20,
    validation_data=(xdev, ydev),
    callbacks=[tensorboard_callback]
)

Epoch 1/20
484/484 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - RootMeanSquaredError: 1.5926 - loss: 2.5898 - val_RootMeanSquaredError: 0.9343 - val_loss: 0.8732
Epoch 2/20
484/484 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - RootMeanSquaredError: 0.9039 - loss: 0.8173 - val_RootMeanSquaredError: 0.8498 - val_loss: 0.7224
Epoch 3/20
484/484 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - RootMeanSquaredError: 0.8391 - loss: 0.7042 - val_RootMeanSquaredError: 0.8203 - val_loss: 0.6732
Epoch 4/20
484/484 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - RootMeanSquaredError: 0.8109 - loss: 0.6576 - val_RootMeanSquaredError: 0.7977 - val_loss: 0.6366
Epoch 5/20
484/484 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - RootMeanSquaredError: 0.7886 - loss: 0.6219 - val_RootMeanSquaredError: 0.7787 - val_loss: 0.6066
Epoch 6/20
484/484 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - RootMeanSquaredError: 0.7697 - loss: 0.5926 - val_RootMeanSquaredError: 0.7623 - val_loss: 0.5814
Epoch 7/20
484/484 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - RootMeanSquaredError: 0.7536 - los

### Launch Tensorboard

In [ ]:
# tensorboard --logdir logs/fit

In [9]:
test_logdir = log_dir
writer = tf.summary.create_file_writer(str(test_logdir))
with writer.as_default():
    for step in range(1, 1000 + 1):
        tf.summary.scalar("my_scalar", np.sin(step / 10), step=step)

        data = (np.random.randn(100) + 2) * step / 100  # gets larger
        tf.summary.histogram("my_hist", data, buckets=50, step=step)

        images = np.random.rand(2, 32, 32, 3) * step / 1000  # gets brighter
        tf.summary.image("my_images", images, step=step)

        texts = ["The step is " + str(step), "Its square is " + str(step**2)]
        tf.summary.text("my_text", texts, step=step)

        sine_wave = tf.math.sin(tf.range(12000) / 48000 * 2 * np.pi * step)
        audio = tf.reshape(tf.cast(sine_wave, tf.float32), [1, -1, 1])
        tf.summary.audio("my_audio", audio, sample_rate=48000, step=step)